In [ ]:
import os 
import duckdb
from dotenv import load_dotenv

In [ ]:
# setting the base path 
base_path = os.getenv('base_path')
database = os.getenv('database_path')

In [3]:
# setting an in-memory database. 
ddb = duckdb.connect(f"{database}/database.db")

In [4]:
# ingest geolocation csv 
ddb.execute(
    """
        CREATE TABLE IF NOT EXISTS geoloc_table AS
        SELECT 
            *
        FROM 
            read_csv_auto(?);
    """, 
    (f"{base_path}/datasets/olist_geolocation_dataset.csv",)
)

In [5]:
# Renaming columns. 
ddb.sql(
    '''
        ALTER TABLE geoloc_table 
        RENAME geolocation_zip_code_prefix TO GeoLocZipCodePrefix;
    '''
)

ddb.sql(
    '''
        ALTER TABLE geoloc_table 
        RENAME geolocation_lat TO GeoLocLat;
    '''
)

ddb.sql(
    '''
        ALTER TABLE geoloc_table 
        RENAME geolocation_lng TO GeoLocLng;
    '''
)

ddb.sql(
    '''
        ALTER TABLE geoloc_table 
        RENAME geolocation_city TO GeoLocCity;
    '''
)

ddb.sql(
    '''
        ALTER TABLE geoloc_table 
        RENAME geolocation_state TO GeoLocState;
    '''
)

BinderException: Binder Error: Table "geoloc_table" does not have a column with name "geolocation_zip_code_prefix"

In [6]:
ddb.sql(
    '''
        SELECT *
        FROM geoloc_table;
    '''
)

┌─────────────────────┬─────────────────────┬─────────────────────┬────────────┬─────────────┐
│ GeoLocZipCodePrefix │      GeoLocLat      │      GeoLocLng      │ GeoLocCity │ GeoLocState │
│       varchar       │       double        │       double        │  varchar   │   varchar   │
├─────────────────────┼─────────────────────┼─────────────────────┼────────────┼─────────────┤
│ 01037               │  -23.54562128115268 │  -46.63929204800168 │ sao paulo  │ SP          │
│ 01046               │ -23.546081127035535 │  -46.64482029837157 │ sao paulo  │ SP          │
│ 01046               │  -23.54612896641469 │  -46.64295148361138 │ sao paulo  │ SP          │
│ 01041               │   -23.5443921648681 │  -46.63949930627844 │ sao paulo  │ SP          │
│ 01035               │ -23.541577961711493 │  -46.64160722329613 │ sao paulo  │ SP          │
│ 01012               │ -23.547762303364266 │  -46.63536053788448 │ são paulo  │ SP          │
│ 01047               │ -23.546273112412678 │  -46

In [7]:
# Checking the table, clean and ensure its integrity - check null values
ddb.sql(
    '''
        SELECT
            COUNT(*) AS total_rows,
            COUNT(CASE WHEN GeoLocZipCodePrefix IS NULL THEN 1 END) AS GeoLocZipCodePrefix_nulls,
            COUNT(CASE WHEN GeoLocLat IS NULL THEN 1 END) AS GeoLocLat_nulls,
            COUNT(CASE WHEN GeoLocLng IS NULL THEN 1 END) AS GeoLocLng_nulls,
            COUNT(CASE WHEN GeoLocCity IS NULL THEN 1 END) AS GeoLocCity,
            COUNT(CASE WHEN GeoLocState IS NULL THEN 1 END) AS GeoLocState,
        FROM 
            geoloc_table
    '''
)

┌────────────┬───────────────────────────┬─────────────────┬─────────────────┬────────────┬─────────────┐
│ total_rows │ GeoLocZipCodePrefix_nulls │ GeoLocLat_nulls │ GeoLocLng_nulls │ GeoLocCity │ GeoLocState │
│   int64    │           int64           │      int64      │      int64      │   int64    │    int64    │
├────────────┼───────────────────────────┼─────────────────┼─────────────────┼────────────┼─────────────┤
│    1000163 │                         0 │               0 │               0 │          0 │           0 │
└────────────┴───────────────────────────┴─────────────────┴─────────────────┴────────────┴─────────────┘

In [8]:
# Checking the table, clean and ensure its integrity - ensuring data quality and integrity
ddb.sql(
    '''
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'geoloc_table';
    '''
)

ddb.sql(
    '''
        SELECT 
            CAST(GeoLocZipCodePrefix AS INT) AS GeoLocZipCodePrefix,
            CAST(GeoLocLat AS DOUBLE) AS GeoLocLat,
            CAST(GeoLocLng AS DOUBLE) AS GeoLocLng,
            CAST(GeoLocCity AS VARCHAR) AS GeoLocCity,
            CAST(GeoLocState AS VARCHAR) AS GeoLocState,
        FROM geoloc_table;
    '''
)

┌─────────────────────┬─────────────────────┬─────────────────────┬────────────┬─────────────┐
│ GeoLocZipCodePrefix │      GeoLocLat      │      GeoLocLng      │ GeoLocCity │ GeoLocState │
│        int32        │       double        │       double        │  varchar   │   varchar   │
├─────────────────────┼─────────────────────┼─────────────────────┼────────────┼─────────────┤
│                1037 │  -23.54562128115268 │  -46.63929204800168 │ sao paulo  │ SP          │
│                1046 │ -23.546081127035535 │  -46.64482029837157 │ sao paulo  │ SP          │
│                1046 │  -23.54612896641469 │  -46.64295148361138 │ sao paulo  │ SP          │
│                1041 │   -23.5443921648681 │  -46.63949930627844 │ sao paulo  │ SP          │
│                1035 │ -23.541577961711493 │  -46.64160722329613 │ sao paulo  │ SP          │
│                1012 │ -23.547762303364266 │  -46.63536053788448 │ são paulo  │ SP          │
│                1047 │ -23.546273112412678 │  -46

In [9]:
ddb.sql(
    '''
        SELECT *
        FROM geoloc_table;
    '''
)

┌─────────────────────┬─────────────────────┬─────────────────────┬────────────┬─────────────┐
│ GeoLocZipCodePrefix │      GeoLocLat      │      GeoLocLng      │ GeoLocCity │ GeoLocState │
│       varchar       │       double        │       double        │  varchar   │   varchar   │
├─────────────────────┼─────────────────────┼─────────────────────┼────────────┼─────────────┤
│ 01037               │  -23.54562128115268 │  -46.63929204800168 │ sao paulo  │ SP          │
│ 01046               │ -23.546081127035535 │  -46.64482029837157 │ sao paulo  │ SP          │
│ 01046               │  -23.54612896641469 │  -46.64295148361138 │ sao paulo  │ SP          │
│ 01041               │   -23.5443921648681 │  -46.63949930627844 │ sao paulo  │ SP          │
│ 01035               │ -23.541577961711493 │  -46.64160722329613 │ sao paulo  │ SP          │
│ 01012               │ -23.547762303364266 │  -46.63536053788448 │ são paulo  │ SP          │
│ 01047               │ -23.546273112412678 │  -46

In [15]:
# Define the output directory and ensure it exists
output_dir = "/Users/macintoshcider/Documents/Programming/Python/ETL/climate-commerce/analytics"
output_file = os.path.join(output_dir, "dim_geolocation.csv")
ddb.execute(
    f"""
        COPY geoloc_table TO '{output_file}' (FORMAT CSV, HEADER);
    """
)
print(f"Data successfully exported to {output_file}")

Data successfully exported to /Users/macintoshcider/Documents/Programming/Python/ETL/climate-commerce/analytics/dim_geolocation.csv
